# Getting Started with Promptolution

## Welcome to Promptolution! 

Discover a powerful tool for evolving and optimizing your LLM prompts. This notebook provides a friendly introduction to Promptolution's core functionality.

We're excited to have you try Promptolution - let's get started!

## Installation
Install Promptolution with a single command

In [10]:
# ! pip install promptolution[api]
%load_ext autoreload
%autoreload 2

INFO 05-14 14:24:03 [importing.py:17] Triton not installed or not compatible; certain GPU-related functions will not be available.
WARNING 05-14 14:24:03 [importing.py:28] Triton is not installed. Using dummy decorators. Install it via `pip install triton` to enable kernelcompilation.
INFO 05-14 14:24:03 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 05-14 14:24:06 [__init__.py:239] Automatically detected platform cuda.
INFO 05-14 14:24:06 [__init__.py:239] Automatically detected platform cuda.
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Imports

In [11]:
import pandas as pd
from promptolution import ExperimentConfig, run_experiment
import nest_asyncio
nest_asyncio.apply() # Required for notebook environments

## Setting Up Your Experiment

### Prepare the data

Below, we're using a subsample of the subjectivity dataset from Hugging Face as an example. When using your own dataset, simply ensure you name the input column "x" and the target column "y", and provide a brief description of your task, that will parsed to the meta-llm during optimization.

In [12]:
df = pd.read_csv("hf://datasets/tasksource/subjectivity/train.csv").sample(500)
df = df.rename(columns={"Sentence": "x", "Label": "y"})
df = df.replace({"OBJ": "objective", "SUBJ": "subjective"})

task_description = "The dataset contains sentences labeled as either subjective or objective. "\
        "The task is to classify each sentence as either subjective or objective. " \
        "The class mentioned first in the response of the LLM will be the prediction."

### Creating Inital Prompts

We've defined some starter prompts below, but feel free to experiment! You might also want to explore create_prompts_from_samples to automatically generate initial prompts based on your data.

In [13]:
init_prompts = [
    'Classify the given text as either an objective or subjective statement based on the tone and language used: e.g. the tone and language used should indicate whether the statement is a neutral, factual summary (objective) or an expression of opinion or emotional tone (subjective). Include the output classes "objective" or "subjective" in the prompt.',
    'What kind of statement is the following text: [Insert text here]? Is it <objective_statement> or <subjective_statement>?',
    'Identify whether a sentence is objective or subjective by analyzing the tone, language, and underlying perspective. Consider the emotion, opinion, and bias present in the sentence. Are the authors presenting objective facts or expressing a personal point of view? The output will be either "objective" (output class: objective) or "subjective" (output class: subjective).',
    'Classify the following sentences as either objective or subjective, indicating the name of the output classes: [input sentence]. Output classes: objective, subjective',
    '_query a text about legal or corporate-related issues, and predict whether the tone is objective or subjective, outputting the corresponding class "objective" for non-subjective language or "subjective" for subjective language_',
    'Classify a statement as either "subjective" or "objective" based on whether it reflects a personal opinion or a verifiable fact. The output classes to include are "objective" and "subjective".',
    'Classify the text as objective or subjective based on its tone and language.',
    'Classify the text as objective or subjective based on the presence of opinions or facts. Output classes: objective, subjective.',
    'Classify the given text as objective or subjective based on its tone, focusing on its intention, purpose, and level of personal opinion or emotional appeal, with outputs including classes such as objective or subjective.',
    "Categorize the text as either objective or subjective, considering whether it presents neutral information or expresses a personal opinion/bias.\n\nObjective: The text has a neutral tone and presents factual information about the actions of Democrats in Congress and the union's negotiations.\n\nSubjective: The text has a evaluative tone and expresses a positive/negative opinion/evaluation about the past performance of the country.",
    'Given a sentence, classify it as either "objective" or "subjective" based on its tone and language, considering the presence of third-person pronouns, neutral language, and opinions. Classify the output as "objective" if the tone is neutral and detached, focusing on facts and data, or as "subjective" if the tone is evaluative, emotive, or biased.',
    'Identify whether the given sentence is subjective or objective, then correspondingly output "objective" or "subjective" in the form of "<output class>, (e.g. "objective"), without quotes. Please note that the subjective orientation typically describes a sentence where the writer expresses their own opinion or attitude, whereas an objective sentence presents facts or information without personal involvement or bias. <output classes: subjective, objective>'
]

### Configure Your LLM

Promptolution offers three flexible ways to access language models:

1. Local LLMs (using the Transformers library)
1. vLLM backend (for efficient serving of large language models)
1. API-based LLMs (compatible with any provider following the OpenAI standard)

For this demonstration, we'll use the DeepInfra API, but you can easily switch to other providers like Anthropic or OpenAI by simply changing the base_url and llm string in the configuration.

In [14]:
# token = "YOUR_API_KEY" # Replace with your Promptolution API key
# load from deepinfratoken.txt
with open("../deepinfratoken.txt", "r") as f:
    token = f.read().strip()

Here's an explanation of each configuration parameter in the ExperimentConfig:

- `task_description`: A string describing the task you're optimizing prompts for. This is used to provide the meta-llm with context about your task.
- `prompts`: A list of initial prompt strings that will be used as the starting point for optimization.
- `n_steps`: The number of optimization steps to run. Higher values allow more exploration and refinement but require more API calls and computational resources.
- `optimizer`: The algorithm used for prompt optimization. Currently we support "evopromptga", "evopromptde", and "opro".
- `api_url`: The API endpoint URL used to access the language model. This example uses DeepInfra's API which follows the OpenAI standard.
- `llm`: The LLM to use for the experiment, as both downstream and meta LLM.
- `token`: Your API authentication token required to access the language model service.

In [15]:
config = ExperimentConfig(
    task_description=task_description,
    prompts=init_prompts,
    n_steps=10,
    optimizer="capo",
    api_url="https://api.deepinfra.com/v1/openai",
    llm="meta-llama/Meta-Llama-3-8B-Instruct",
    token=token,
)

## Run Your Experiment

With everything configured, you're ready to optimize your prompts! The `run_experiment` function will run the optimization and evaluate on a holdout set. You can expect this cell to take a few minutes to run.

In [16]:
prompts = run_experiment(df, config)

The LLM does not have a tokenizer. Using simple token count.
c:\Users\tzehl\Documents\programming\promptolution\.venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:430: RuntimeWarning: Precision loss occurred in moment calculation due to catastrophic cancellation. This occurs when the data are nearly identical. Results may be unreliable.
  return hypotest_fun_in(*args, **kwds)
Error during optimization step: unsupported operand type(s) for +: 'float' and 'NoneType'
Exiting optimization loop.


As you can see, most optimized prompts are semantically very similar, however they often differ heavily in performance. This is exactly what we observed in our experiments across various LLMs and datasets. Running prompt optimization is an easy way to gain significant performance improvements on your task for free!

If you run into any issues while using Promptolution, please feel free to contact us. We're also happy to receive support through pull requests and other contributions to the project.


Happy prompt optimizing! 🚀✨ We can't wait to see what you build with Promptolution! 🤖💡

In [18]:
prompts

,prompt,score
0,Classify the text as objective or subjective b...,0.733333
1,Classify the text as objective or subjective b...,0.700000
2,Classify the given text as either an objective...,0.666667
3,What kind of statement is the following text: ...,0.666667
4,"Classify a statement as either ""subjective"" or...",0.566667
5,Classify the given text as objective or subjec...,0.566667
6,Identify whether a sentence is objective or su...,0.533333
7,"Given a sentence, classify it as either ""objec...",0.466667
8,Classify the following sentences as either obj...,0.400000
9,Identify whether the given sentence is subject...,0.133333
